---
title: Lab_1
jupyter: python3
format: 
  html:
    embed-resources: true
---


In [137]:
import pandas as pd
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")

Using the skeleton of the pluralize_gift() function, complete the code so that the function takes a gift and returns the appropriate plural.

I used AI for general debugging help, anywhere where serious help was used is listed, as instructed

In [138]:
def pluralize_gift(gift):
  """
  Returns plural of a noun
  
  Parameters
  ----------
  gift: str
    A noun
    
  Return
  ------
  str
    Plural version
  """

  if gift.endswith('y'):
    gift = gift[:-1] + 'ies'
  elif gift.endswith(('s', 'sh', 'ch', 'x', 'z')):
    gift = gift + 'es'
  else:
    gift = gift + 's'

  return gift


In [139]:
# Unit Testing
pluralize_gift("dog")
pluralize_gift("dish")
pluralize_gift("Doggy")

'Doggies'

Try your function out on the smaller and then larger gift data set. Consider: is your function vectorized? If not, how would you run it on all the gifts in the column.

In [140]:
# Should work
pluralize_gift("goose")

# Will work if your function is vectorized! 
# pluralize_gift(xmas['Gift.Item'])
# I made this a comment because it kept messing up the "run All" command later on in my testing

'gooses'

It will not work using a vector, meaning that mapping will be needed for iteration in later steps

Write a function called make_phrase() that takes as input the necessary information, and returns a phrase. For example,

make_phrase(num_word = "ten", 
            item = "lords", 
            verb = "a-leaping", 
            adjective = "", 
            location = "")
should return

"ten lords a-leaping"

In [141]:
def make_phrase(num, num_word, item, verb, adjective, location):
    """
    Makes phrase for the Twelve Days of Christmas song.
    
    Parameters
    ----------
    num : int
        Day number from 1-12 (ex. 10)
    num_word : str
        The number in word form (ex. "ten")
    item : str
        The gift (ex. "lord")
    verb : str
        The verb happening (ex., "a-leaping")
    adjective : str
        The adjective describing the gift (ex. "fluffy)
    location : str
        The location (ex. "in a pear tree")
    
    Returns
    -------
    str
        A phrase combining all pieces
    """
    
    ## Step 1: Replace NAs with blank strings
    verb = "" if pd.isna(verb) else verb
    adjective = "" if pd.isna(adjective) else adjective
    location = "" if pd.isna(location) else location
    
    ## Step 2: If the day number is larger than 1, the gift items need pluralized!
    ### Hint: call the function you created above!
    if num > 1:
        item = pluralize_gift(item)
    
    ## Step 3: Figure out if a gift item starts with a vowel
    starts_with_vowel = item[0].lower() in 'aeiou'
    
    ## Step 4: For the first day, if the gift item starts with a vowel, replace the day with "an" and if the gift item does not start with a vowel, replace the day with "a" (e.g. a partridge in a pear tree). If it is not the first day, use just the number word (e.g. ten lords a leap)
    if num == 1:
        day_word = "an" if starts_with_vowel else "a"
    else:
        day_word = num_word
    
    ## Step 5: Put all of the pieces together into one string and return!
    pieces = [day_word, adjective, item, verb, location]
    phrase = " ".join([p for p in pieces if p != ""])
    
    return phrase


In [142]:
make_phrase(1, "one", "dog", "Jumping", "", "")

'a dog Jumping'

Make sure to try your function out on small examples and on the xmas data.

Then, use the function to make a new column of the xmas column called Full.Phrase containing the sentences for the new gift on that day.

In [143]:
# FIguring out columns for the next function
print(xmas.columns)


Index(['Day', 'Day.in.Words', 'Gift.Item', 'Verb', 'Adjective', 'Location'], dtype='object')


In [144]:
xmas['Full.Phrase'] = xmas.apply(
    lambda row: make_phrase(
        num=row['Day'],
        num_word=row['Day.in.Words'],
        item=row['Gift.Item'],
        verb=row['Verb'],
        adjective=row['Adjective'],
        location=row['Location']
    ),
    axis=1
)


In [145]:
xmas[['Day', 'Gift.Item', 'Full.Phrase']].head()


,Day,Gift.Item,Full.Phrase
0,1,partridge,a partridge in a pear tree
1,2,dove,second turtle doves
2,3,hen,third french hens
3,4,bird,fourth calling birds
4,5,ring,fifth golden rings


I am getting the wrong terms for the full phrase when it comes to the numbers, like second instead of two and third insteda of three, so I am going to map it

In [146]:
map_nums = {
    "first": "one",
    "second": "two",
    "third": "three",
    "fourth": "four",
    "fifth": "five",
    "sixth": "six",
    "seventh": "seven",
    "eighth": "eight",
    "ninth": "nine",
    "tenth": "ten",
    "eleventh": "eleven",
    "twelfth": "twelve"
}

# this includes the code to make NA values empty, rather than causing an error which I had to look up
xmas['Day.in.Words'] = xmas['Day.in.Words'].map(map_nums).fillna(xmas['Day.in.Words'])

In [147]:
xmas['Full.Phrase'] = xmas.apply(
    lambda row: make_phrase(
        num=row['Day'],
        num_word=row['Day.in.Words'],
        item=row['Gift.Item'],
        verb=row['Verb'],
        adjective=row['Adjective'],
        location=row['Location']
    ),
    axis=1
)

In [148]:
xmas[['Day', 'Gift.Item', 'Full.Phrase']].head()

,Day,Gift.Item,Full.Phrase
0,1,partridge,a partridge in a pear tree
1,2,dove,two turtle doves
2,3,hen,three french hens
3,4,bird,four calling birds
4,5,ring,five golden rings


Write a function called sing_day() that takes as input:

A dataset (input as a dataframe)

A number indicating which day to sing about (input as an integer)

The name of a column in the dataset that contains the phrases for each day (input as an tidy name)

For example,

sing_day(xmas, 2, Full.Phrase)
should return

On the second day of Christmas, my true love sent to me:
two turtle doves and
a partridge in a pear tree.

In [149]:
def sing_day(dataset, num, phrase_col):
    """
    Sings the song for the input day of Christmas.

    Parameters
    ----------
    dataset : pd.DataFrame
        DataFrame with the gifts for each day
    num : int
        Which day to sing about (1-12)
    phrase_col : str
        Column name with the full phrases for each gift

    Returns
    -------
    str
        Full song for that day
    """
    
    # Step 1: Setup the intro line
    number_map = {
        1:"first", 2:"second", 3:"third", 4:"fourth", 5:"fifth",
        6:"sixth", 7:"seventh", 8:"eighth", 9:"ninth", 10:"tenth",
        11:"eleventh", 12:"twelfth"
    }
    num_word = number_map[num]
    intro = "On the " + num_word + " day of Christmas, my true love sent to me:"

    # Step 2: Sing the gift phrases
    # Hint: What order are they gifts sung in each day?
    gifts = ""
    for i in range(num, 0, -1):
        phrase = dataset.loc[dataset['Day'] == i, phrase_col].values[0]
        if i == 1 and num > 1:
            phrase = "and " + phrase
        gifts += phrase + "\n"

    # Step 3: Put it all together and return
    verse = intro + "\n" + gifts.rstrip()
    return verse


In [150]:
sing_day(xmas, 10, "Full.Phrase")

'On the tenth day of Christmas, my true love sent to me:\nten lords a-leaping\nnine ladies dancing\neight maids a-milking\nseven swans a-swimming\nsix gooses a-laying\nfive golden rings\nfour calling birds\nthree french hens\ntwo turtle doves\nand a partridge in a pear tree'

This testing line is working weirdly for me, making one long string w \n's to show the different lines

In [151]:
print(sing_day(xmas, 12, 'Full.Phrase'))

On the twelfth day of Christmas, my true love sent to me:
twelve drummers drumming
eleven pipers piping
ten lords a-leaping
nine ladies dancing
eight maids a-milking
seven swans a-swimming
six gooses a-laying
five golden rings
four calling birds
three french hens
two turtle doves
and a partridge in a pear tree


Though this very similar testing line is wokring much better, is this ok?


Run appropriate code to output the lyrics for the entire 12 Days of Christmas song.

In [152]:
# Create a series of days 1 to 12
days = pd.Series(range(1, 13))


In [153]:
# Use .map() to generate each verse
full_song = days.map(lambda day: sing_day(xmas, day, 'Full.Phrase'))

In [154]:
# Optionally, display the lyrics
for verse in full_song:
    print(verse + "\n")

On the first day of Christmas, my true love sent to me:
a partridge in a pear tree

On the second day of Christmas, my true love sent to me:
two turtle doves
and a partridge in a pear tree

On the third day of Christmas, my true love sent to me:
three french hens
two turtle doves
and a partridge in a pear tree

On the fourth day of Christmas, my true love sent to me:
four calling birds
three french hens
two turtle doves
and a partridge in a pear tree

On the fifth day of Christmas, my true love sent to me:
five golden rings
four calling birds
three french hens
two turtle doves
and a partridge in a pear tree

On the sixth day of Christmas, my true love sent to me:
six gooses a-laying
five golden rings
four calling birds
three french hens
two turtle doves
and a partridge in a pear tree

On the seventh day of Christmas, my true love sent to me:
seven swans a-swimming
six gooses a-laying
five golden rings
four calling birds
three french hens
two turtle doves
and a partridge in a pear tree


Then, load the following dataset, and run your code again on this dataset instead to get a surprise song! (The column names and formats of xmas2 are the same as those for xmas.)

In [155]:
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")

In [156]:
map_nums = {
    "first": "one",
    "second": "two",
    "third": "three",
    "fourth": "four",
    "fifth": "five",
    "sixth": "six",
    "seventh": "seven",
    "eighth": "eight",
    "ninth": "nine",
    "tenth": "ten",
    "eleventh": "eleven",
    "twelfth": "twelve"
}

# this includes the code to make NA values empty, rather than causing an error which I had to look up
xmas2['Day.in.Words'] = xmas2['Day.in.Words'].map(map_nums).fillna(xmas2['Day.in.Words'])

In [157]:
xmas2['Full.Phrase'] = xmas2.apply(
    lambda row: make_phrase(
        num=row['Day'],
        num_word=row['Day.in.Words'],
        item=row['Gift.Item'],
        verb=row['Verb'],
        adjective=row['Adjective'],
        location=row['Location']
    ),
    axis=1
)

In [158]:
full_song2 = days.map(lambda day: sing_day(xmas2, day, 'Full.Phrase'))

In [159]:
for verse in full_song2:
    print(verse + "\n")

On the first day of Christmas, my true love sent to me:
an email from Cal Poly

On the second day of Christmas, my true love sent to me:
two meal points
and an email from Cal Poly

On the third day of Christmas, my true love sent to me:
three lost pens
two meal points
and an email from Cal Poly

On the fourth day of Christmas, my true love sent to me:
four course reviews
three lost pens
two meal points
and an email from Cal Poly

On the fifth day of Christmas, my true love sent to me:
five practice exams
four course reviews
three lost pens
two meal points
and an email from Cal Poly

On the sixth day of Christmas, my true love sent to me:
six graders grading
five practice exams
four course reviews
three lost pens
two meal points
and an email from Cal Poly

On the seventh day of Christmas, my true love sent to me:
seven seniors stressing
six graders grading
five practice exams
four course reviews
three lost pens
two meal points
and an email from Cal Poly

On the eighth day of Christmas, 